<a href="https://colab.research.google.com/github/Mattia-Colbertaldo/PrimacyRL/blob/main/MultirunDopamineCartpole.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Reproducibility Notebook for PrimacyRL
> Multiple runs for trend analysis.  
> In contrast to the SingleAgentDopamineCartpole notebook the Agent is now trained n times
---
_Authored by Patrik Rác, Méline Trochon, Valentina Moretti, and Mattia Colbertaldo_

---

The notebook installs the modified _dopamine_ library and trains a simple *DQNAgent* on Cartpole (environment provided by Gym). It

## Step 1: Install Dopamine
---

In [ ]:
# @title Install Dopamine from source
!rm -r dopamine_restart
!git clone -b patrik https://github.com/Mattia-Colbertaldo/dopamine_restart.git
!pip install -r dopamine_restart/requirements.txt

# Install the dopamine package itself (required as there might be previous unseen instalations)
%cd dopamine_restart
!pip install .
%cd ..

rm: cannot remove 'dopamine_restart': No such file or directory
Cloning into 'dopamine_restart'...
remote: Enumerating objects: 4365, done.
remote: Counting objects: 100% (1843/1843), done.
remote: Compressing objects: 100% (537/537), done.
remote: Total 4365 (delta 1434), reused 1410 (delta 1306), pack-reused 2522
Receiving objects: 100% (4365/4365), 21.19 MiB | 19.46 MiB/s, done.
Resolving deltas: 100% (2929/2929), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 858.3/858.3 kB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 781.3/781.3 kB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 61.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 434.7/434.7 kB 44.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for AutoROM.ac

## Step 2: Set-up and run the experiment
---


In [ ]:
# @title Import dependencies and set up base path
import numpy as np
import os

from dopamine.discrete_domains import run_experiment
from dopamine.colab import utils as colab_utils
from absl import flags
import gin.tf

BASE_PATH = '/tmp/colab_dopamine_run'  # @param

/usr/local/lib/python3.10/dist-packages/tensorflow/python/framework/dtypes.py:35: DeprecationWarning: ml_dtypes.float8_e4m3b11 is deprecated. Use ml_dtypes.float8_e4m3b11fnuz
  from tensorflow.tsl.python.lib.core import pywrap_ml_dtypes
/usr/local/lib/python3.10/dist-packages/gin/tf/__init__.py:48: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <
/usr/local/lib/python3.10/dist-packages/flax/configurations.py:42: DeprecationWarning: jax.config.define_bool_state is deprecated. Please use other libraries for configuration instead.
  return jax_config.define_bool_state('flax_' + name, default, help)
/usr/local/lib/python3.10/dist-packages/chex/_src/pytypes.py:53: DeprecationWarning: jax.core.Shape is deprecated. Use Shape = Sequence[int | Any].
  Shape = jax.core.Shape
/usr/local/lib/python3.10/dist-packages/chex/_src/pytypes.py:54: DeprecationWarning: jax.random.KeyArray is deprecated. Use j

In [ ]:
# @title Set up the hyperparameters for the run

# Modified from dopamine/agents/dqn/config/dqn_cartpole.gin
dqn_config = """
# Hyperparameters for a simple DQN-style Cartpole agent. The hyperparameters
# chosen achieve reasonable performance.
import dopamine.discrete_domains.gym_lib
import dopamine.discrete_domains.run_experiment
import dopamine.agents.dqn.dqn_agent
import dopamine.replay_memory.circular_replay_buffer
import gin.tf.external_configurables

DQNAgent.observation_shape = %gym_lib.CARTPOLE_OBSERVATION_SHAPE
DQNAgent.observation_dtype = %gym_lib.CARTPOLE_OBSERVATION_DTYPE
DQNAgent.stack_size = %gym_lib.CARTPOLE_STACK_SIZE
DQNAgent.network = @gym_lib.CartpoleDQNNetwork
DQNAgent.gamma = 0.6
DQNAgent.update_horizon = 10
DQNAgent.min_replay_history = 500
DQNAgent.update_period = 8
DQNAgent.target_update_period = 10
DQNAgent.epsilon_fn = @dqn_agent.identity_epsilon
DQNAgent.tf_device = '/gpu:*'  # use '/cpu:*' for non-GPU version
DQNAgent.optimizer = @tf.train.AdamOptimizer()
DQNAgent.reset_dense1 = False
DQNAgent.reset_dense2 = True
DQNAgent.reset_last_layer = True
DQNAgent.reset_period= 10000000000
DQNAgent.reset_max = 3

tf.train.AdamOptimizer.learning_rate = 0.001
tf.train.AdamOptimizer.epsilon = 0.0003125

create_gym_environment.environment_name = 'CartPole'
create_gym_environment.version = 'v1'
create_agent.agent_name = 'dqn'
TrainRunner.create_environment_fn = @gym_lib.create_gym_environment
Runner.num_iterations = 120
Runner.training_steps = 500
Runner.evaluation_steps = 100
Runner.max_steps_per_episode = 10000  # Default max episode length.

WrappedReplayBuffer.replay_capacity = 50000
WrappedReplayBuffer.batch_size = 128
"""
gin.parse_config(dqn_config, skip_unknown=False)

([],
 ['dopamine.discrete_domains.gym_lib',
  'dopamine.discrete_domains.run_experiment',
  'dopamine.agents.dqn.dqn_agent',
  'dopamine.replay_memory.circular_replay_buffer',
  'gin.tf.external_configurables'])

In [ ]:
# @title Train DQN on Cartpole
n_runs = 10

from absl import logging
logging.set_verbosity('info')

log_files = list()

for i in range(n_runs):
  LOG_PATH = BASE_PATH + str(i)
  log_files.append(LOG_PATH)
  dqn_runner = run_experiment.create_runner(LOG_PATH, schedule='continuous_train_and_eval')
  print('Will train DQN agent number {}, please be patient, may be a while...'.format(i))
  dqn_runner.run_experiment()
  print('Done training agent number {}!'.format(i))

## Step 3: Load the experiment logs and plot results
---

In [ ]:
# @title Load experiment logs

import collections

parameter_set = collections.OrderedDict([
    ('agent', ['DQN+reset 2']),
    ('game', ['Cartpole'])
])

data = colab_utils.read_experiment(log_files[0], verbose=True,
                                      parameter_set=parameter_set,
                              summary_keys=['train_episode_returns'])

data['agent'] = 'DQN+reset 2'
data['run'] = 1

for i in range(1,len(log_files)):
  tmp_data = colab_utils.read_experiment(log_files[i], verbose=True,
                                      parameter_set=parameter_set,
                              summary_keys=['train_episode_returns'])
  tmp_data['agent'] = 'DQN+reset 2'
  tmp_data['run'] = i+1
  data = data.merge(tmp_data, how='outer')

In [ ]:
# @title Plot the results
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


fig, ax = plt.subplots(figsize=(16,8))
sns.lineplot(x='iteration', y='train_episode_returns', hue='agent',
              data=data, ax=ax)
plt.yscale('log')
plt.grid()
plt.title('Cartpole')
# plt.savefig("demo.png")
plt.show()